In [12]:
# Install dependencies
!pip install torch==1.10.0 torchvision==0.11.1
!pip install yolov5


ERROR: Could not find a version that satisfies the requirement torch==1.10.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0)
ERROR: No matching distribution found for torch==1.10.0

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

C:\Users\Admin\Desktop\python\yolov5\yolov5\yolov5


Cloning into 'yolov5'...
C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# Download the COCO dataset (train2017, val2017, and annotations)
import os
import requests
from zipfile import ZipFile


In [15]:
# Create directories for the dataset
os.makedirs("coco/train2017", exist_ok=True)
os.makedirs("coco/val2017", exist_ok=True)
os.makedirs("coco/annotations", exist_ok=True)

In [16]:
# Function to download and extract dataset
def download_and_extract(url, extract_to='.'):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    with ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(local_filename)


In [ ]:
# Download images and annotations
download_and_extract('http://images.cocodataset.org/zips/train2017.zip', 'coco/train2017')
download_and_extract('http://images.cocodataset.org/zips/val2017.zip', 'coco/val2017')
download_and_extract('http://images.cocodataset.org/annotations/annotations_trainval2017.zip', 'coco/annotations')


In [ ]:
# Define the dataset paths
train_images_path = "coco/train2017"
val_images_path = "coco/val2017"
train_annotations_path = "coco/annotations/instances_train2017.json"
val_annotations_path = "coco/annotations/instances_val2017.json"


In [ ]:
# Define the YOLOv5 configuration content
coco_config = f"""
train: {train_images_path}
val: {val_images_path}

# number of classes
nc: 80

# class names
names: ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]
"""


In [ ]:
# Save the configuration to a file
with open('coco.yaml', 'w') as f:
    f.write(coco_config)


In [ ]:
# Train the YOLOv5 model
!python train.py --img 640 --batch 16 --epochs 10 --data coco.yaml --weights yolov5s.pt


In [ ]:
# List the contents of the 'runs/train' directory
training_runs_dir = 'runs/train/'
runs = sorted(os.listdir(training_runs_dir), key=lambda x: os.path.getctime(os.path.join(training_runs_dir, x)))

# Get the latest run directory
latest_run_dir = os.path.join(training_runs_dir, runs[-1])
print("Latest run directory:", latest_run_dir)

# Path to the best weights file
best_weights_path = os.path.join(latest_run_dir, 'weights', 'best.pt')
print("Best weights path:", best_weights_path)


In [ ]:
# Perform inference and visualization
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=best_weights_path)

# Path to an individual image for inference testing
test_image_path = "coco/val2017/000000000139.jpg"  # Replace with the path to your test image

# Perform inference
img = Image.open(test_image_path)
results = model(img)

# Visualize the predictions
results.show()

# Optionally, save the predictions
results.save()